# Keras Callbacks and Functional API

In [ ]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
(X_train_t, y_train), (X_test_t, y_test) = cifar10.load_data()

X_train_t = X_train_t.astype('float32') / 255.
X_test_t = X_test_t.astype('float32') / 255.

X_train = X_train_t.reshape(len(X_train_t), 32*32*3)
X_test = X_test_t.reshape(len(X_test_t), 32*32*3)

In [ ]:
print("Training set:")
print("Tensor images shape:\t", X_train_t.shape)
print("Flat images shape:\t", X_train.shape)
print("Labels shape:\t\t", y_train.shape)

In [ ]:
plt.figure(figsize=(15, 4))
for i in range(0, 8):
    plt.subplot(1, 8, i+1)
    plt.imshow(X_train[i].reshape(32, 32, 3))
    plt.title(y_train[i, 0])

## Callbacks on a simple model

In [ ]:
outpath='/tmp/tensorboard/cifar/'

early_stopper = EarlyStopping(monitor='val_accuracy', patience=10)
tensorboard = TensorBoard(outpath, histogram_freq=1)
checkpointer = ModelCheckpoint(outpath+'weights_epoch_{epoch:02d}_val_accuracy_{val_accuracy:.2f}.hdf5',
                               monitor='val_acc')

In [ ]:
model = Sequential([
    Dense(1024, activation='relu', input_dim=3072),
    Dense(512, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_split=0.1,
          callbacks=[early_stopper,
                     tensorboard,
                     checkpointer])

In [ ]:
import os
sorted(os.listdir(outpath))

Now check the tensorboard.

- If using provided instance, just browse to: `http://<your-domain>/tensorboard/`

- If using local, open a terminal, activate the environment and run:
```
tensorboard --logdir=/tmp/tensorboard/cifar/
```
then open a browser at `localhost:6006`

You should see something like this:

![tensorboard.png](../assets/tensorboard.png)

If you can't access port 6006, you can use Local Tunel. Open another terminal and run [localtunnel](https://localtunnel.github.io/www/) on port 6006:
```
lt --port 6006
```
3. Go to the url provided

You should see something like this.

> TIP: if you get an error `lt: command not found` install localtunnel as:
```
sudo npm install -g localtunnel
```

## Exercise 1: Keras functional API

We'e built a model using the `Sequential API` from tensorflow.keras. Keras also offers a [functional API](https://keras.io/getting-started/functional-api-guide/). This API is the way to go for defining complex models, such as multi-output models, directed acyclic graphs, or models with shared layers.

Can you rewrite the model above using the functional API?

In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

In [ ]:
inputs = Input(shape=(3072,))
x = Dense(1024, activation='relu')(inputs)
x = Dense(512, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

model = Model(inputs=inputs, outputs=predictions)

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_split=0.1)

# Final test evaluation
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

## Exercise 2: Convolutional Model with Functional API

The above model is a very simple fully connected deep neural network. As we have seen, Convolutional Neural Networks are much more powerful when dealing with images. The original data has shape:

    (N_images, Height, Width, Channels)
    
Can you write a convolutional model using the functional API?

Bonus points if you use callbacks to write data to tensorboard.

In [ ]:
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Conv2D, MaxPool2D, AveragePooling2D, Flatten

In [ ]:
outpath_conv = outpath.replace('cifar', 'cifar_conv')

In [ ]:
tensorboard_conv = TensorBoard(outpath_conv, histogram_freq=1)

In [ ]:
inputs = Input(shape=(32, 32, 3))
x = Conv2D(32, (3, 3),
           padding='same',
           activation='relu')(inputs)
x = Conv2D(32, (3, 3), activation='relu')(x)
x = MaxPool2D(pool_size=(2, 2))(x)

x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPool2D(pool_size=(2, 2))(x)

x = Flatten()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

model = Model(inputs=inputs, outputs=predictions)

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


# Fit your model
model.fit(X_train_t, y_train,
          batch_size=32,
          epochs=10,
          validation_split=0.1,
          shuffle=True,
          verbose=1,
          callbacks=[tensorboard_conv]
          )

# Final test evaluation
score = model.evaluate(X_test_t, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

## Exrcise 3: Discuss with the person next to you 

1. What are the pros/cons of the sequential API?
- What are the pros/cons of the functional API?
- What are the key differences between a Fully connected and a Convolutional neural network?
- What is a dropout layer? How does it work? Why does it help?
